In [1]:
import tensorflow as tf
import os

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
(train_image, train_lable),_ = tf.keras.datasets.fashion_mnist.load_data()

In [4]:
train_image = train_image/255

In [5]:
train_lable.shape

(60000,)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10))

In [7]:
optimizer = tf.keras.optimizers.Adam()

In [8]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [10]:
def train_step(model,images, labels):
    #计算梯度 计算损失函数与可model的变量训练参数的梯度
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss(model, images, labels) #记录损失函数的值
    grads = t.gradient(loss_step, model.trainable_variables)  # 计算损失函数与可model的变量训练参数的梯度
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) #优化 改变可训练参数变量的值
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [11]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy') #初始化计算对象

In [12]:
cp_dir = './customtrain_cp'
cp_prefix = os.path.join(cp_dir, 'ckpt') #设置保存前缀

In [13]:
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 model=model
) #定义保存的内容

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_lable))

In [15]:
dataset = dataset.shuffle(10000).batch(32)

In [21]:
def train():
    for epoch in range(5):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch{} loss is {}, accuracy is {}'.format(epoch,
                                                         train_loss.result(),
                                                         train_accuracy.result()))
        
        train_loss.reset_states() #重置对象
        train_accuracy.reset_states()
        
        #每隔两个epoch，进行保存
        if (epoch + 1)%2 ==0:
            checkpoint.save(file_prefix = cp_prefix)

In [22]:
train()

Epoch0 loss is 0.5004569888114929, accuracy is 0.8249833583831787
Epoch1 loss is 0.37520208954811096, accuracy is 0.8643500208854675
Epoch2 loss is 0.33405163884162903, accuracy is 0.8780666589736938
Epoch3 loss is 0.31028130650520325, accuracy is 0.8857499957084656
Epoch4 loss is 0.29134881496429443, accuracy is 0.8921499848365784


In [16]:
checkpoint.restore(tf.train.latest_checkpoint(cp_dir)) #恢复最新的检查点

In [17]:
tf.argmax(model(train_image, training=False),axis=-1).numpy()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([9, 0, 0, ..., 3, 0, 5], dtype=int64)

In [18]:
train_lable

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [25]:
(tf.argmax(model(train_image, training=False),axis=-1).numpy() == train_lable).sum()/len(train_lable) #查看正确率

0.8849666666666667

In [ ]:
#0.8849666666666667